In [1]:
from copy import deepcopy, copy
from datetime import datetime
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
import qiskit as qk
from qiskit.circuit.library import MCMT

from context import tools as t

plt.style.use('science')

In [2]:
def get_filename_png():
    now = datetime.now()
    return now.strftime('figures/%Y-%m-%d--%H:%M:%S-')+NB_NAME[:-6]+'.png'

def get_filename_txt():
    now = datetime.now()
    return now.strftime('data/%Y-%m-%d--%H:%M:%S-')+NB_NAME[:-6]+'.txt'

def circ(n_qubits):
    qr = qk.QuantumRegister(n_qubits, 'q')
    qc = qk.QuantumCircuit(qr)
    return qr, qc

def sort(dic):
    return sorted(dic.items())

def isdagger(gate, dag_gate):
    qr = qk.QuantumRegister(gate.num_qubits, 'q')
    qc = qk.QuantumCircuit(qr)
    qc.compose(gate, qr, inplace=True)
    qc.compose(dag_gate, qr, inplace=True)
    qc.measure_all()
    counts = t.counts(qc)
    return list(counts.keys()) == ['0000000000']

def all_combinations(s):
    arr = np.array(list(s), dtype=object)
    dots_pos = np.where(arr == '.')[0]
    bit_lst = [list(''.join(str(bit) for bit in bits)) for bits in product([0, 1], repeat=len(dots_pos))]
    result = []
    for bit in bit_lst:
        arr[dots_pos] = bit
        result.append(''.join(str(b) for b in arr))
    return result

In [3]:
def get_U():
    # build W|0> = |who>
    qr, qc = circ(6)
    qc.h([0, 1, 3])
    qc.cnot([0, 1, 3], [2, 5, 4])
    W_gate = deepcopy(qc)

    # build T|0> = |talks> = (|00>+|11>)/sqrt(2)
    # build A|0> = |answers> = (|00>+|11>)/sqrt(2)
    qr, qc = circ(2)
    qc.h(0)
    qc.cnot(0, 1)
    T_gate = deepcopy(qc)
    A_gate = deepcopy(qc)

    # build U|0>=|psi>
    qr, qc = circ(10)
    qc.compose(W_gate, qr[:6], inplace=True)
    qc.compose(T_gate, qr[6:8], inplace=True)
    qc.compose(A_gate, qr[8:10], inplace=True)
    # qc.barrier()

    # contraction 1
    qc.cnot(5, 6)
    qc.h(5)
    # qc.barrier()
    # contraction 2
    qc.cnot(4, 7)
    qc.h(4)
    # qc.barrier()
    # contraction 3
    qc.cnot(1, 8)
    qc.h(1)
    # qc.barrier()
    # contraction 4
    qc.cnot(0, 9)
    qc.h(0)
    # qc.barrier()

    U_circ = deepcopy(qc)
#     print(U_circ.draw())

    U_gate = U_circ.to_gate(label='U')
    return U_gate

In [4]:
def get_Q(U_gate, desired_str):
    n_qubits = U_gate.num_qubits

    # build Udag_gate
    Udag_gate = U_gate.reverse_ops()
    assert(isdagger(U_gate, Udag_gate))

    # build O_gate (oracle)
    assert(len(desired_str)==n_qubits)

    desired0 = [i for i,s in enumerate(desired_str) if s=='0']
    desired1 = [i for i,s in enumerate(desired_str) if s=='1']
    affected = [i for i,s in enumerate(desired_str) if s!='.']

    qr, O_circ= circ(n_qubits+1)
    O_circ.x(qr[desired0])
    O_circ.compose(MCMT('cx', len(affected), 1), qr[affected] + [qr[-1]], inplace=True)
    O_circ.x(qr[desired0])

    O_gate = O_circ.to_gate(label='O')

    # build S0_gate (that flips sign of |0> state)
    qr, S0_circ = circ(n_qubits)
    S0_circ.x(qr)
    S0_circ.h(-1)
    S0_circ.compose(MCMT('cx', n_qubits-1, 1), qr, inplace=True)
    S0_circ.h(-1)
    S0_circ.x(qr)

    S0_gate = S0_circ.to_gate(label='S0')

    # build Q_gate (grover operator)
    qr, Q_circ = circ(n_qubits+1)
    Q_circ.compose(O_gate, qr, inplace=True)
    Q_circ.compose(Udag_gate, qr[:-1], inplace=True)
    Q_circ.compose(S0_gate, qr[:-1], inplace=True)
    Q_circ.compose(U_gate, qr[:-1], inplace=True)

    Q_gate = Q_circ.to_gate(label='Q')
    return Q_gate

In [19]:
def get_Q_powers():
    U_gate = get_U()
    Q_gate = get_Q(U_gate, '00.1000000')
    qr, qc = circ(Q_gate.num_qubits)
    qc.compose(Q_gate, qr, inplace=True)
    power = 1
    
    while True:
        yield qc.to_gate(label=f'Q^{power}').control()
        print(power)
        power += 1
        qc.compose(Q_gate, qr, inplace=True)
        
U_gate = get_U()
n_qubits = U_gate.num_qubits

qr = qk.QuantumRegister(2*n_qubits+1, 'q')
cr = qk.ClassicalRegister(n_qubits, 'c')
qc = qk.QuantumCircuit(qr, cr)

qc.h(qr[:n_qubits])
qc.compose(U_gate, qr[n_qubits:-1], inplace=True)

g = get_Q_powers()
l = len(qr)
wires = lambda i: qr[i:-l+i+1] + qr[n_qubits:]

for i in range(n_qubits-1, -1, -1):
    qc.compose(next(g), wires(i), inplace=True)

print(qc.draw())

1
2
3
4
5
6
7
8
9
      ┌───┐                                                                »
 q_0: ┤ H ├────────────────────────────────────────────────────────────────»
      ├───┤                                                                »
 q_1: ┤ H ├────────────────────────────────────────────────────────────────»
      ├───┤                                                                »
 q_2: ┤ H ├────────────────────────────────────────────────────────────────»
      ├───┤                                                                »
 q_3: ┤ H ├───────────────────────────────────────────────────────────■────»
      ├───┤                                                           │    »
 q_4: ┤ H ├──────────────────────────────────────────────────■────────┼────»
      ├───┤                                                  │        │    »
 q_5: ┤ H ├─────────────────────────────────────────■────────┼────────┼────»
      ├───┤                                         │     

In [34]:
qc.h(qr[:n_qubits])
print(qc.draw())

      ┌───┐                                                                 »
 q_0: ┤ H ├─────────────────────────────────────────────────────────────────»
      ├───┤                                                                 »
 q_1: ┤ H ├─────────────────────────────────────────────────────────────────»
      ├───┤                                                                 »
 q_2: ┤ H ├─────────────────────────────────────────────────────────────────»
      ├───┤                                                                 »
 q_3: ┤ H ├─────────────────────────────────────────────────────────────────»
      ├───┤                                                                 »
 q_4: ┤ H ├────────────────────────────────────────────────────────────■────»
      ├───┤                                                            │    »
 q_5: ┤ H ├───────────────────────────────────────────────────■────────┼────»
      ├───┤                                          ┌───┐    │ 

In [27]:
import pickle

In [28]:
def save_circ(qc, filename):
    with open(filename, 'wb') as out:
        pickle.dump(qc, out, pickle.HIGHEST_PROTOCOL)

In [35]:
save_circ(qc, 'circuits/estimation_who_talks.pkl')

In [30]:
with open('cenas', 'rb') as inp:
    circ = pickle.load(inp)

In [31]:
circ.draw()

┌───┐                                                                »
 q_0: ┤ H ├────────────────────────────────────────────────────────────────»
      ├───┤                                                                »
 q_1: ┤ H ├────────────────────────────────────────────────────────────────»
      ├───┤                                                                »
 q_2: ┤ H ├────────────────────────────────────────────────────────────────»
      ├───┤                                                                »
 q_3: ┤ H ├───────────────────────────────────────────────────────────■────»
      ├───┤                                                           │    »
 q_4: ┤ H ├──────────────────────────────────────────────────■────────┼────»
      ├───┤                                                  │        │    »
 q_5: ┤ H ├─────────────────────────────────────────■────────┼────────┼────»
      ├───┤                                         │        │        │    »
 q_6: ┤ H ├────────────────────────────────■────────┼────────┼────────┼────»
      ├───┤                                │        │        │        │    »
 q_7: ┤ H ├───────────────────────■────────┼────────┼────────┼────────┼────»
      ├───┤                       │        │        │        │        │    »
 q_8: ┤ H ├──────────────■────────┼────────┼────────┼────────┼────────┼────»
      ├───┤              │        │        │        │        │        │    »
 q_9: ┤ H ├─────■────────┼────────┼────────┼────────┼────────┼────────┼────»
      ├───┴┐┌───┴───┐┌───┴───┐┌───┴───┐┌───┴───┐┌───┴───┐┌───┴───┐┌───┴───┐»
q_10: ┤0   ├┤0      ├┤0      ├┤0      ├┤0      ├┤0      ├┤0      ├┤0      ├»
      │    ││       ││       ││       ││       ││       ││       ││       │»
q_11: ┤1   ├┤1      ├┤1      ├┤1      ├┤1      ├┤1      ├┤1      ├┤1      ├»
      │    ││       ││       ││       ││       ││       ││       ││       │»
q_12: ┤2   ├┤2      ├┤2      ├┤2      ├┤2      ├┤2      ├┤2      ├┤2      ├»
      │    ││       ││       ││       ││       ││       ││       ││       │»
q_13: ┤3   ├┤3      ├┤3      ├┤3      ├┤3      ├┤3      ├┤3      ├┤3      ├»
      │    ││       ││       ││       ││       ││       ││       ││       │»
q_14: ┤4   ├┤4      ├┤4      ├┤4      ├┤4      ├┤4      ├┤4      ├┤4      ├»
      │  U ││       ││       ││       ││       ││       ││       ││       │»
q_15: ┤5   ├┤5  Q^1 ├┤5  Q^2 ├┤5  Q^3 ├┤5  Q^4 ├┤5  Q^5 ├┤5  Q^6 ├┤5  Q^7 ├»
      │    ││       ││       ││       ││       ││       ││       ││       │»
q_16: ┤6   ├┤6      ├┤6      ├┤6      ├┤6      ├┤6      ├┤6      ├┤6      ├»
      │    ││       ││       ││       ││       ││       ││       ││       │»
q_17: ┤7   ├┤7      ├┤7      ├┤7      ├┤7      ├┤7      ├┤7      ├┤7      ├»
      │    ││       ││       ││       ││       ││       ││       ││       │»
q_18: ┤8   ├┤8      ├┤8      ├┤8      ├┤8      ├┤8      ├┤8      ├┤8      ├»
      │    ││       ││       ││       ││       ││       ││       ││       │»
q_19: ┤9   ├┤9      ├┤9      ├┤9      ├┤9      ├┤9      ├┤9      ├┤9      ├»
      └────┘│       ││       ││       ││       ││       ││       ││       │»
q_20: ──────┤10     ├┤10     ├┤10     ├┤10     ├┤10     ├┤10     ├┤10     ├»
            └───────┘└───────┘└───────┘└───────┘└───────┘└───────┘└───────┘»
c: 10/═════════════════════════════════════════════════════════════════════»
                                                                           »
«                                  
« q_0: ──────────────────────■─────
«                            │     
« q_1: ─────────────■────────┼─────
«                   │        │     
« q_2: ────■────────┼────────┼─────
«          │        │        │     
« q_3: ────┼────────┼────────┼─────
«          │        │        │     
« q_4: ────┼────────┼────────┼─────
«          │        │        │     
« q_5: ────┼────────┼────────┼─────
«          │        │        │     
« q_6: ────┼────────┼────────┼─────
«          │        │        │     
«

In [32]:
print(circ.draw())

      ┌───┐                                                                »
 q_0: ┤ H ├────────────────────────────────────────────────────────────────»
      ├───┤                                                                »
 q_1: ┤ H ├────────────────────────────────────────────────────────────────»
      ├───┤                                                                »
 q_2: ┤ H ├────────────────────────────────────────────────────────────────»
      ├───┤                                                                »
 q_3: ┤ H ├───────────────────────────────────────────────────────────■────»
      ├───┤                                                           │    »
 q_4: ┤ H ├──────────────────────────────────────────────────■────────┼────»
      ├───┤                                                  │        │    »
 q_5: ┤ H ├─────────────────────────────────────────■────────┼────────┼────»
      ├───┤                                         │        │        │    »